In [1]:
import json
import os
import shutil

import torch
import yaml

from transformers import OwlViTForObjectDetection

/ext3/miniconda3/envs/owl_boto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.dataset import get_dataloaders

In [22]:
import importlib

import src
importlib.reload(src.utils)
from src.utils import paco_to_owl_box, BoxUtil
from torchvision.ops import box_convert as _box_convert

In [23]:
def get_training_config():
    with open("config.yaml", "r") as stream:
        data = yaml.safe_load(stream)
        return data["training"]

In [24]:
training_cfg = get_training_config()

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [26]:
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32").to(device)

In [27]:
train_dataloader, test_dataloader = get_dataloaders(batch_size=5)

In [28]:
batch1 = next(train_dataloader.__iter__())

In [29]:
batch1_inputs = batch1[0]

In [30]:
batch1_inputs["attention_mask"] = batch1_inputs["attention_mask"].view(-1, 16)
batch1_inputs["input_ids"] = batch1_inputs["input_ids"].view(-1, 16)

In [31]:
outputs = model(**batch1_inputs.to(device))

In [32]:
outputs.logits.shape

torch.Size([5, 576, 3])

In [33]:
b = paco_to_owl_box(batch1[1][:, None, :], batch1[2])